In [1]:
import sys
import pickle

import numpy as np
import random
    
from vgg_face import VGG16, preprocess_input
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, EarlyStopping

from keras import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import load_model

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

In [3]:
def calculate_metrics(y_pred, y_true):
    acc = sum(y_pred == y_true) / len(y_pred)
    # Index of real images
    ind_gen = y_true == 1
    # Index of GAN images
    ind_imp = y_true == 0
    
    num_real = sum(ind_gen)
    num_gan = sum(ind_imp)
    # From the real images how many predicted as GAN
    num_missly_gan = sum(y_pred[ind_gen] == 0)
    # From the gan images how many predicted as real
    num_missly_real = sum(y_pred[ind_imp] == 1)
    
    print("False Acceptance Rate(FAR): {}".format(100 * num_missly_gan / num_gan))
    
    print("False Rejectance Rate(FRR): {}".format(100 *num_missly_real / num_real))
    
    print("Accuracy: {}".format(100 * acc))

In [4]:
from keras.models import load_model

vgg_model = load_model('trained_vgg.h5')

C:\Users\Burak\AppData\Local\Programs\Python\Python36\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
tester_model.summary()

NameError: name 'tester_model' is not defined

In [5]:
image_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,validation_split=0.2)

In [6]:
train_generator = image_datagen.flow_from_directory(
  'train_128',
  target_size=(128, 128),
  color_mode='rgb',
  batch_size=32,
  class_mode='categorical',
  shuffle=True,
  subset='training'
)

validation_generator = image_datagen.flow_from_directory(
  'train_128',
  target_size=(128, 128),
  color_mode='rgb',
  batch_size=32,
  class_mode='categorical',
  subset='validation'
)

Found 8100 images belonging to 2 classes.
Found 2025 images belonging to 2 classes.


In [7]:
data = []
labels = []
gen_iter = 0
max_iter = train_generator.samples // train_generator.batch_size
for d, l in train_generator:
    data.append(vgg_model.predict(d))
    labels.append(np.argmax(l, axis=1))
    gen_iter += 1
    if gen_iter == max_iter:
        break

In [8]:
data = np.array(data).reshape((-1,512))
labels = np.array(labels).reshape(-1)

In [9]:
val_data = []
val_labels = []
gen_iter = 0
max_iter = validation_generator.samples // validation_generator.batch_size
for d, l in validation_generator:
    val_data.append(vgg_model.predict(d))
    val_labels.append(np.argmax(l, axis=1))
    gen_iter += 1
    if gen_iter == max_iter:
        break

In [10]:
val_data = np.array(val_data).reshape((-1,512))
val_labels = np.array(val_labels).reshape(-1)

In [11]:
test_generator = image_datagen.flow_from_directory(
  'test_128',
  target_size=(128, 128),
  color_mode='rgb',
  batch_size=32,
  class_mode='categorical',
)

Found 5183 images belonging to 2 classes.


In [12]:
test_data = []
test_labels = []
gen_iter = 0
max_iter = test_generator.samples // test_generator.batch_size
for d, l in test_generator:
    test_data.append(vgg_model.predict(d))
    test_labels.append(np.argmax(l, axis=1))
    gen_iter += 1
    if gen_iter == max_iter:
        break

In [13]:
test_data = np.array(test_data).reshape((-1,512))
test_labels = np.array(test_labels).reshape(-1)

In [91]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=75, max_depth=20, random_state=0)
clf.fit(data, labels)
predictions = clf.predict(val_data)
calculate_metrics(predictions, val_labels)

predictions = clf.predict(test_data)
calculate_metrics(predictions, test_labels)


False Acceptance Rate(FAR): 9.028459273797841
False Rejectance Rate(FRR): 6.41925777331996
Accuracy: 92.26190476190477
False Acceptance Rate(FAR): 9.45216049382716
False Rejectance Rate(FRR): 6.015625
Accuracy: 92.25543478260869


In [92]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(data, labels)
predictions = clf.predict(val_data)
calculate_metrics(predictions, val_labels)

predictions = clf.predict(test_data)
calculate_metrics(predictions, test_labels)


False Acceptance Rate(FAR): 6.280667320902846
False Rejectance Rate(FRR): 5.2156469408224675
Accuracy: 94.24603174603175
False Acceptance Rate(FAR): 6.404320987654321
False Rejectance Rate(FRR): 4.921875
Accuracy: 94.33229813664596


In [42]:
from sklearn import svm
clf = svm.SVC(kernel="linear", C=0.025)
clf.fit(data, labels)
predictions = clf.predict(val_data)
calculate_metrics(predictions, val_labels)

False Acceptance Rate(FAR): 6.366307541625857
False Rejectance Rate(FRR): 6.231155778894473
Accuracy: 93.70039682539682


In [93]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(data, labels)
predictions = clf.predict(val_data)
calculate_metrics(predictions, val_labels)

predictions = clf.predict(test_data)
calculate_metrics(predictions, test_labels)


False Acceptance Rate(FAR): 9.126594700686947
False Rejectance Rate(FRR): 7.021063189568706
Accuracy: 91.91468253968253
False Acceptance Rate(FAR): 9.760802469135802
False Rejectance Rate(FRR): 6.6796875
Accuracy: 91.77018633540372


In [16]:
data.shape

(8096, 512)

In [32]:
model = Sequential()
model.add(Dense(64, input_dim=512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
adam = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=3e-4, amsgrad=False)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
output_model_name = 'classifier.h5'
checkpoint = ModelCheckpoint(output_model_name,
                             monitor='val_acc', verbose=1,
                             save_best_only=True, save_weights_only=False,
                             mode='auto', period=1)

In [34]:
num_iteration_train = train_generator.n // train_generator.batch_size
num_iteration_val = validation_generator.n // validation_generator.batch_size

In [35]:
image_preds = model.fit(x= data, y=labels, epochs=50, callbacks=[checkpoint], validation_data=(val_data, val_labels))

Train on 8096 samples, validate on 2016 samples
Epoch 1/50
8096/8096 [==============================] - 2s 291us/step - loss: 0.3209 - acc: 0.8828 - val_loss: 0.1901 - val_acc: 0.9271

Epoch 00001: val_acc improved from -inf to 0.92708, saving model to classifier.h5
Epoch 2/50
8096/8096 [==============================] - 2s 203us/step - loss: 0.1493 - acc: 0.9411 - val_loss: 0.1865 - val_acc: 0.9375

Epoch 00002: val_acc improved from 0.92708 to 0.93750, saving model to classifier.h5
Epoch 3/50
8096/8096 [==============================] - 2s 226us/step - loss: 0.1252 - acc: 0.9512 - val_loss: 0.1761 - val_acc: 0.9350

Epoch 00003: val_acc did not improve from 0.93750
Epoch 4/50
8096/8096 [==============================] - 2s 197us/step - loss: 0.1035 - acc: 0.9600 - val_loss: 0.1772 - val_acc: 0.9360

Epoch 00004: val_acc did not improve from 0.93750
Epoch 5/50
8096/8096 [==============================] - 2s 280us/step - loss: 0.0882 - acc: 0.9653 - val_loss: 0.1847 - val_acc: 0.9405



8096/8096 [==============================] - 2s 205us/step - loss: 0.0105 - acc: 0.9956 - val_loss: 0.4194 - val_acc: 0.9355

Epoch 00043: val_acc did not improve from 0.94246
Epoch 44/50
8096/8096 [==============================] - 2s 204us/step - loss: 0.0066 - acc: 0.9969 - val_loss: 0.4711 - val_acc: 0.9400

Epoch 00044: val_acc did not improve from 0.94246
Epoch 45/50
8096/8096 [==============================] - 2s 206us/step - loss: 0.0073 - acc: 0.9969 - val_loss: 0.4343 - val_acc: 0.9390

Epoch 00045: val_acc did not improve from 0.94246
Epoch 46/50
8096/8096 [==============================] - 2s 197us/step - loss: 0.0077 - acc: 0.9972 - val_loss: 0.4569 - val_acc: 0.9390

Epoch 00046: val_acc did not improve from 0.94246
Epoch 47/50
8096/8096 [==============================] - 2s 202us/step - loss: 0.0093 - acc: 0.9963 - val_loss: 0.4274 - val_acc: 0.9360

Epoch 00047: val_acc did not improve from 0.94246
Epoch 48/50
8096/8096 [==============================] - 2s 216us/step -

In [36]:
pred = np.array(model.predict_on_batch(test_data) > 0.5).astype(int)

In [37]:
calculate_metrics(pred.reshape(-1), test_labels)

False Acceptance Rate(FAR): 6.947124662292551
False Rejectance Rate(FRR): 4.373291682936353
Accuracy: 94.33229813664596
